# Taller SummIT IA - 2025

**Autor:** Jazna Meza Hidalgo

**Correo Electrónico:** ja.meza@profesor.duoc.cl

**Fecha de Creación:** Junio 2025

**Versión:** 1.0  


---

## Descripción


Este notebook es el taller del evento SummIT-IA versión 2025.

---


In [2]:
!pip install transformers torch torchvision pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


In [26]:
import gradio as gr
import numpy as np
import cv2
import requests
import torch
from io import BytesIO
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [ ]:
# Cargar modelo preentrenado
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")
modelo = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")

In [27]:
def distorsionar_imagen(imagen):
    img_np = np.array(imagen)

    # Desenfoque fuerte
    blur = cv2.GaussianBlur(img_np, (25, 25), 0)

    # Agregar ruido (simula artefactos)
    ruido = blur + np.random.normal(0, 30, img_np.shape).astype(np.uint8)
    ruido = np.clip(ruido, 0, 255)

    return Image.fromarray(ruido)


In [28]:
# Función de predicción que devuelve imagen, clase predicha, confianza y distribución completa
def detectar_imagen_fake_completo(imagen, url):
    try:
        if url:
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            imagen_procesada = Image.open(BytesIO(response.content)).convert("RGB")
        elif imagen:
            imagen_procesada = imagen.convert("RGB")
        else:
            return None, None, "", "", "⚠️ Debes subir una imagen o ingresar una URL."

        # Nueva imagen distorsionada
        imagen_distorsionada = distorsionar_imagen(imagen_procesada)

        resultados = []
        for img in [imagen_procesada, imagen_distorsionada]:
            inputs = processor(images=img, return_tensors="pt")
            with torch.no_grad():
                outputs = modelo(**inputs)
                logits = outputs.logits
                probs = torch.nn.functional.softmax(logits, dim=-1)[0]

            predicted_class = torch.argmax(probs).item()
            etiqueta = modelo.config.id2label[predicted_class]
            probabilidad = probs[predicted_class].item()
            resumen = f"{etiqueta} ({probabilidad:.2%})"

            detalles = "\n".join([
                f"{modelo.config.id2label[i]}: {probs[i].item():.2%}" for i in range(len(probs))
            ])

            resultados.append((resumen, detalles))

        # Retorna: original, distorsionada, resumen1, resumen2, comparativo
        return (
            imagen_procesada,
            imagen_distorsionada,
            resultados[0][0],
            resultados[1][0],
            f"🧠 Original:\n{resultados[0][1]}\n\n🌀 Distorsionada:\n{resultados[1][1]}"
        )

    except Exception as e:
        return None, None, "", "", f"❌ Error al procesar la imagen: {str(e)}"


In [29]:
iface = gr.Interface(
    fn=detectar_imagen_fake_completo,
    inputs=[
        gr.Image(type="pil", label="Sube una imagen o pega una URL"),
        gr.Textbox(label="...o pega la URL de una imagen")
    ],
    outputs=[
        gr.Image(type="pil", label="🖼 Imagen original"),
        gr.Image(type="pil", label="🌀 Imagen distorsionada"),
        gr.Textbox(label="Predicción original"),
        gr.Textbox(label="Predicción distorsionada"),
        gr.Textbox(label="📋 Comparación completa")
    ],
    title="Detector de Rostros Falsos con Vision Transformer",
    description="Sube una imagen o usa una URL para detectar si un rostro es real o generado por IA. También se muestra una versión distorsionada.",
    submit_btn=gr.Button("Predecir"),
    clear_btn=gr.Button("Limpiar")
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5cf66c09c976484494.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
